In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

%load_ext autoreload
%autoreload 2
from runner import run

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        
            
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(128 * 7 * 7, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [14]:
# print out number of parameters
model = VGG16(10)
num_params = sum(p.numel() for p in model.parameters())
print(f'Number of parameters: {num_params}')

Number of parameters: 14161482


In [6]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to fit AlexNet input dimensions.
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalization parameters for ImageNet.
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
from torch.utils.data import DataLoader

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

hparams = {
    "lr": 3e-4,
    "num_classes": 10,
    "batch_size": 32,
    "epochs": 10
}

train_loader = DataLoader(trainset, batch_size=hparams["batch_size"] , shuffle=True)
val_loader = DataLoader(testset, batch_size=hparams["batch_size"], shuffle=True)

model = VGG16(num_classes=hparams['num_classes']).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hparams["lr"])

run(hparams, model, "vgg16-cifar10", criterion, optimizer, val_loader, train_loader, device)


Epoch 1:  15%|█▌        | 237/1563 [02:25<13:35,  1.63batch/s, loss=2.09, val_loss=1.77, val_acc=tensor(0.4102, device='mps:0')] 


KeyboardInterrupt: 